In [23]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, StringType, upper

In [24]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [25]:
SQLContext = SparkSession.builder.master("local[1]") \
                .appName("session-0") \
                .getOrCreate()

In [26]:
data_path = "/mnt/chaseproject/uva/kd5eyn/x509/2022-04-30/anon.x509_20220430_0000-0030-0400.log.gz"
x509_df = SQLContext.read.parquet(data_path)


In [27]:
x509_df.columns

['_lpp_ver',
 'anon_san_dns',
 'anon_san_ip',
 'anon_san_uri',
 'basic_constraints_ca',
 'certificate_curve',
 'certificate_exponent',
 'certificate_issuer',
 'certificate_key_alg',
 'certificate_key_length',
 'certificate_key_type',
 'certificate_not_valid_after',
 'certificate_not_valid_before',
 'certificate_serial',
 'certificate_sig_alg',
 'certificate_subject',
 'certificate_version',
 'client_cert',
 'fingerprint',
 'host_cert',
 'san_dns',
 'san_email',
 'san_ip',
 'san_uri',
 'ts']

In [28]:
x509_df.collect()[0]["certificate_issuer"]

'CN=5Vx2Cd,OU=Server CA 1B,O=Amazon,C=US'

In [29]:
ssl_data = "/mnt/chaseproject/uva/kd5eyn/ssl/2022-04-30/anon.ssl_20220430_0000-0030-0400.log.gz"
ssl_df = SQLContext.read.parquet(ssl_data)

In [30]:
ssl_df.columns

['_lpp_ver',
 'anon_orig',
 'anon_resp',
 'cert_chain_fps',
 'cipher',
 'client_cert_chain_fps',
 'curve',
 'established',
 'id_orig_h',
 'id_orig_p',
 'id_resp_h',
 'id_resp_p',
 'ja3',
 'ja3s',
 'last_alert',
 'next_protocol',
 'resumed',
 'server_name',
 'sni_matches_cert',
 'ssl_history',
 'ts',
 'uid',
 'validation_status',
 'version']

In [31]:
ssl_df = ssl_df.withColumn("leaf_cert_fp", ssl_df.cert_chain_fps[0])

In [32]:
import pandas as pd
ssl_df_panda = ssl_df.sample(fraction=0.001).toPandas()
pd.set_option('display.max_columns', None)
ssl_df_panda

,_lpp_ver,anon_orig,anon_resp,cert_chain_fps,cipher,client_cert_chain_fps,curve,established,id_orig_h,id_orig_p,id_resp_h,id_resp_p,ja3,ja3s,last_alert,next_protocol,resumed,server_name,sni_matches_cert,ssl_history,ts,uid,validation_status,version,leaf_cert_fp
0,1.11-7,uva,none,[172bf6e6294816ac36c14384e708503b618c4a4d9bc6d...,TLS_ECDHE_RSA_WITH_AES_128_GCM_SHA256,[],x25519,True,128.143.27.194,38944,40.126.23.162,443,da12c94da8021bbaf502907ad086e7bc,dd638b91d791c45c599b83addf922232,None,None,False,graph.microsoft.com,True,CsxknGIi,2022-04-30T03:59:59.436410Z,CurmUr1S4u60hMRxel,ok,TLSv12,172bf6e6294816ac36c14384e708503b618c4a4d9bc6db...
1,1.11-7,uva,none,[8d0825c06ea8eff432db0d972e379d15593c6b885dd10...,TLS_ECDHE_ECDSA_WITH_AES_256_GCM_SHA384,[],x25519,True,137.54.242.44,64080,17.248.191.101,443,773906b0efdefa24a7f2b8eb6985bf37,9b340edddb1203ae20167d3089fa10af,None,http/1.1,False,p55-caldav.icloud.com,True,CsxuknGIi,2022-04-30T04:00:00.541566Z,CylfVE3PEpewWPDSf3,ok,TLSv12,8d0825c06ea8eff432db0d972e379d15593c6b885dd10f...
2,1.11-7,none,uva,[31d106f8e763efdeac282708b2159fdef7ee1187ab4fa...,TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384,[2e99aeb3e642f21e96b8cdf38a8f732181771cb70d1e7...,x25519,True,12.215.234.52,64350,128.143.40.8,20017,9e1a4e14e04fa642b15649052ed150f1,ec74a5c51106f0419184d0dd08fb05bc,None,None,False,2hs6T2CK.virginia.edu,True,CsxkrnXGYIti,2022-04-30T04:00:00.513963Z,Ceryu74I33pMyK01nc,ok,TLSv12,31d106f8e763efdeac282708b2159fdef7ee1187ab4fa6...
3,1.11-7,uva,none,[058465fab5e708890810862988d8f9549bf9f56311850...,TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384,[],secp256r1,True,199.111.54.37,11149,23.207.53.148,443,773906b0efdefa24a7f2b8eb6985bf37,d93a04e792b59c119912bd0f60915165,None,http/1.1,False,c.apple.news,True,CsxuknGIi,2022-04-30T04:00:01.109413Z,CX4pTu1NUHZHePDSI,ok,TLSv12,058465fab5e708890810862988d8f9549bf9f563118504...
4,1.11-7,uva,none,None,TLS_AES_256_GCM_SHA384,None,x25519,True,199.111.54.27,22955,173.223.163.213,443,656b9a2f4de6ed4909e157482860ab3d,15af977ce25de452b96affa2addb1036,None,None,False,weather-data.apple.com,None,CsiI,2022-04-30T04:00:01.098519Z,CZc5lnFMUkf6WZqd7,None,TLSv13,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2912,1.11-7,uva,none,None,TLS_AES_128_GCM_SHA256,None,x25519,True,137.54.126.9,50392,172.217.13.238,443,cd08e31494f9531f560d64c695473da9,eb1d94daa7e0344597e756a1fb6e7054,None,None,False,play.google.com,None,CsiI,2022-04-30T04:29:57.531390Z,CiZaHp1LMVu1HF9wf5,None,TLSv13,None
2913,1.11-7,uva,none,[b437268fddb33a91d413114b826c886d1c27ea980ab0a...,TLS_ECDHE_RSA_WITH_AES_128_GCM_SHA256,[],secp256r1,True,199.111.55.60,3566,3.225.80.92,443,773906b0efdefa24a7f2b8eb6985bf37,704239182a9091e4453fdbfe0fd17586,None,None,False,home.hulu.com,True,CsxknGIi,2022-04-30T04:29:57.810891Z,CjQBcmgZOh6zHuo2c,ok,TLSv12,b437268fddb33a91d413114b826c886d1c27ea980ab0a6...
2914,1.11-7,uva,none,None,TLS_ECDHE_RSA_WITH_AES_128_GCM_SHA256,None,None,True,199.111.54.117,32539,162.125.6.19,443,effe9b59e99e730d14f23d971080682b,5b94af9bf6efc9dea416841602004fbb,None,h2,True,api.dropboxapi.com,None,CsiI,2022-04-30T04:29:57.905232Z,CSwVSLYCYMyvcgpJ3,None,TLSv12,None
2915,1.11-7,uva,none,None,None,None,None,False,199.111.55.14,11187,34.95.67.231,443,1aad9aff2523e91bbf4e8276a5b2140e,None,None,None,False,fcm.holmesmind.com,None,C,2022-04-30T04:29:53.230837Z,CAyaxY13b61TbkNmu7,None,None,None


In [33]:
x509_df_pandas = x509_df.sample(fraction=0.001).toPandas()
x509_df_pandas

,_lpp_ver,anon_san_dns,anon_san_ip,anon_san_uri,basic_constraints_ca,certificate_curve,certificate_exponent,certificate_issuer,certificate_key_alg,certificate_key_length,certificate_key_type,certificate_not_valid_after,certificate_not_valid_before,certificate_serial,certificate_sig_alg,certificate_subject,certificate_version,client_cert,fingerprint,host_cert,san_dns,san_email,san_ip,san_uri,ts
0,1.11-7,"[none, none, none, none, none, none, none, non...",None,None,False,None,65537,CN=270O4FqybK 1Owx9vLZ6aBl 7fKG1uTi5z 1V - 3B2...,rsaEncryption,2048,rsa,2023-04-10T05:31:30.000000Z,2022-03-09T06:31:31.000000Z,F773E75E5176ED1BC519081B,sha256WithRSAEncryption,"CN=*.march05.sparta.3g.qq.com,O=Shenzhen Tence...",3,False,b306fb500a4a096057ddc034557558d16b7163a85774dd...,True,"[*.march05.sparta.3g.qq.com, bi.wifi.html5.qq....",None,None,None,2022-04-30T04:15:27.611888Z
1,1.11-7,[none],None,None,False,prime256v1,None,"CN=EGgSB0Ksdp 2ZF 1qL 1V-7,O=Cloudflare\, Inc....",id-ecPublicKey,256,ecdsa,2022-06-08T03:59:59.000000Z,2021-06-08T04:00:00.000000Z,F684CA34BB98F799DE149A134FD74DA1,ecdsa-with-SHA256,"CN=venmo.zendesk.com,O=Cloudflare\, Inc.,L=San...",3,False,63fe8b01222c3c88450ba62a94ed2c08cfb26b43cbb724...,True,[venmo.zendesk.com],None,None,None,2022-04-30T04:15:44.661189Z
2,1.11-7,"[none, none]",None,None,False,None,65537,"CN=3v 2P3XD 4KPuUO 2hTX11qddI1 57JOJKKDi - 3s,...",rsaEncryption,2048,rsa,2022-08-29T04:39:23.000000Z,2020-06-13T18:58:19.000000Z,7C0789AA83023270,sha256WithRSAEncryption,"CN=*.cognitoforms.com,O=Cognito\, LLC,L=Columb...",3,False,7cdcf43897c30e76ad0340642aa1234228b5ded1550640...,True,"[*.cognitoforms.com, cognitoforms.com]",None,None,None,2022-04-30T04:16:28.502792Z
3,1.11-7,[none],None,None,False,None,65537,"CN=eyFPEBq4 79F 24o 3B2Kqa 5gvA 14i,O=DigiCert...",rsaEncryption,2048,rsa,2022-05-25T03:59:59.000000Z,2021-05-07T04:00:00.000000Z,92C96A4762472314AD39FFD8A91D4A6D,sha256WithRSAEncryption,"CN=AMCPEpicCE.ah.org,O=Adventist Health System...",3,False,bb2cd03afe2528fe25f625c2df5e74e770dd174daa7970...,True,[AMCPEpicCE.ah.org],None,None,None,2022-04-30T04:24:50.765511Z


In [34]:
ssl_df_panda["leaf_cert_fp"]

0       172bf6e6294816ac36c14384e708503b618c4a4d9bc6db...
1       8d0825c06ea8eff432db0d972e379d15593c6b885dd10f...
2       31d106f8e763efdeac282708b2159fdef7ee1187ab4fa6...
3       058465fab5e708890810862988d8f9549bf9f563118504...
4                                                    None
                              ...                        
2912                                                 None
2913    b437268fddb33a91d413114b826c886d1c27ea980ab0a6...
2914                                                 None
2915                                                 None
2916    00c25c0330ed25cac56b3edcbe37ddd365e6e1b36a7759...
Name: leaf_cert_fp, Length: 2917, dtype: object

In [35]:
issuer_df = ssl_df.select(["leaf_cert_fp"])
issuer_df = issuer_df.dropDuplicates().na.drop()
issuer_df = issuer_df.withColumnRenamed(existing="leaf_cert_fp", new="fingerprint")

In [36]:
def get_cert_issuer(ssl_leaf_cert: str, x509_df: pyspark.sql.DataFrame) -> str:
    cert_row = x509_df.filter(issuer_df.leaf_cert == x509_df.certificate_serial)
    if cert_row.count() < 1:
        return "No Issuer Found"
    return str(cert_row.certificate_issuer)

In [37]:
issuer_df = issuer_df.join(x509_df, on="fingerprint", how="inner")
issuer_df.count()

2193

In [38]:
issuer_df = issuer_df.select(["fingerprint", "certificate_issuer"])
issuer_df_panda = issuer_df.toPandas()

In [39]:
issuer_df_panda.iloc[0]["fingerprint"]

'90b03afec858010493087c3044be99caedd1985d6c48a02be1db5c623e8261a7'

In [40]:
def get_issuer_column(issuer_df: pd.DataFrame, leaf_cert_fp: str):
    issuer_row = issuer_df[issuer_df["fingerprint"] == leaf_cert_fp]
    if len(issuer_row) == 0:
        return "No Issuer"
    return str(issuer_row["certificate_issuer"].astype("string"))
    

In [41]:
fingerprint = "90b03afec858010493087c3044be99caedd1985d6c48a02be1db5c623e8261a7"
get_issuer_column(issuer_df_panda, fingerprint)

'0    CN=5Vx2Cd,OU=Server CA 1B,O=Amazon,C=US\nName: certificate_issuer, dtype: string'

In [45]:
ssl_df = ssl_df.join(issuer_df, ssl_df.leaf_cert_fp == issuer_df.fingerprint, how="inner")
ssl_df.count()

33649392

In [46]:
ssl_df.sample(fraction=0.001).toPandas()

,_lpp_ver,anon_orig,anon_resp,cert_chain_fps,cipher,client_cert_chain_fps,curve,established,id_orig_h,id_orig_p,id_resp_h,id_resp_p,ja3,ja3s,last_alert,next_protocol,resumed,server_name,sni_matches_cert,ssl_history,ts,uid,validation_status,version,leaf_cert_fp,fingerprint,certificate_issuer,fingerprint,certificate_issuer
0,1.11-7,uva,none,[63064ed215c65b4df8cd59ecb23c2382296a39d788878...,TLS_ECDHE_RSA_WITH_AES_128_GCM_SHA256,[],x25519,True,199.111.54.66,26080,203.107.55.19,443,9b02ebd3a43b62d825e1ac605b621dc8,00447ab319e9d94ba2b4c1248e155917,None,http/1.1,False,devs-data.dutils.com,True,CsxknGIti,2022-04-30T04:22:27.965285Z,CGqWeW2K9jyROiyGYj,ok,TLSv12,63064ed215c65b4df8cd59ecb23c2382296a39d7888789...,63064ed215c65b4df8cd59ecb23c2382296a39d7888789...,"CN=6eKx1VEn 79F 67 24o 7OrsR 3B2Kqa 5gvA 1V-7,...",63fbe8d03c76b01dec9adb33e00c01f138b17d3fe94f1c...,"CN=57,O=Let's Encrypt,C=US"
1,1.11-7,uva,none,[90b03afec858010493087c3044be99caedd1985d6c48a...,TLS_ECDHE_RSA_WITH_AES_128_GCM_SHA256,[],x25519,True,199.111.54.250,6638,99.86.231.71,443,a0e9f5d64349fb13191bc781f81f42e1,6f84bbe9810ec4ea9061cc1a02eaf83c,None,None,False,locales.mspcdns.com,True,CsxknGIti,2022-04-30T04:04:41.096432Z,CFJNUl3PXGKGjuGdia,ok,TLSv12,90b03afec858010493087c3044be99caedd1985d6c48a0...,90b03afec858010493087c3044be99caedd1985d6c48a0...,"CN=5Vx2Cd,OU=Server CA 1B,O=Amazon,C=US",288fb2cf6bb766fdd6ee284ddd0aa42d4db341385de6a0...,"CN=57,O=Let's Encrypt,C=US"
2,1.11-7,uva,none,[63064ed215c65b4df8cd59ecb23c2382296a39d788878...,TLS_ECDHE_RSA_WITH_AES_128_GCM_SHA256,[],x25519,True,199.111.54.66,26057,203.107.55.19,443,9b02ebd3a43b62d825e1ac605b621dc8,00447ab319e9d94ba2b4c1248e155917,None,http/1.1,False,devs-data.dutils.com,True,CsxknGIti,2022-04-30T04:22:29.929133Z,Cgg9Od2PiLulTes97c,ok,TLSv12,63064ed215c65b4df8cd59ecb23c2382296a39d7888789...,63064ed215c65b4df8cd59ecb23c2382296a39d7888789...,"CN=6eKx1VEn 79F 67 24o 7OrsR 3B2Kqa 5gvA 1V-7,...",e443cecfbd8006d96cb997151c46d5f4bca6e3d8709798...,"CN=57,O=Let's Encrypt,C=US"
3,1.11-7,uva,none,[90b03afec858010493087c3044be99caedd1985d6c48a...,TLS_ECDHE_RSA_WITH_AES_128_GCM_SHA256,[],x25519,True,199.111.54.250,6603,99.84.111.83,443,a0e9f5d64349fb13191bc781f81f42e1,beb7069ae409bccfed702c17ad004223,None,None,False,content.mspcdns.com,True,CsxuknGIti,2022-04-30T04:01:53.170445Z,CkMiVE2jyu5uq76pc4,ok,TLSv12,90b03afec858010493087c3044be99caedd1985d6c48a0...,90b03afec858010493087c3044be99caedd1985d6c48a0...,"CN=5Vx2Cd,OU=Server CA 1B,O=Amazon,C=US",d053ae02ddc788a697be437f853e8f3efda9a1e7db63a2...,"CN=5Vx2Cd,OU=Server CA 1B,O=Amazon,C=US"
4,1.11-7,uva,none,[90b03afec858010493087c3044be99caedd1985d6c48a...,TLS_ECDHE_RSA_WITH_AES_128_GCM_SHA256,[],x25519,True,199.111.54.249,6063,99.84.111.83,443,a0e9f5d64349fb13191bc781f81f42e1,beb7069ae409bccfed702c17ad004223,None,None,False,content.mspcdns.com,True,CsxuknGIti,2022-04-30T04:06:04.600405Z,CgHB5q4b4xJmULO0Mf,ok,TLSv12,90b03afec858010493087c3044be99caedd1985d6c48a0...,90b03afec858010493087c3044be99caedd1985d6c48a0...,"CN=5Vx2Cd,OU=Server CA 1B,O=Amazon,C=US",ee50e9cb171c886d29bdda567b4d0fb3060a086e31d386...,"CN=57,O=Let's Encrypt,C=US"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33652,1.11-7,uva,none,[9cbabc969a489d90986ee4dfd642a5caff0c2decf98b2...,TLS_ECDHE_ECDSA_WITH_AES_128_GCM_SHA256,[5e7970e9e73521806f1755456f899939b0066aa8568c7...,x25519,True,199.111.55.16,9627,47.42.75.199,41054,3a72da99311dbe67317350129fc32cea,778c9f321f5eb60d83b1c28712ab7e42,None,None,False,None,None,cSXKRNxgyiTI,2022-04-30T04:17:35.651602Z,ChAyMy4dEaKxx1M2ld,self signed certificate,DTLSv12,9cbabc969a489d90986ee4dfd642a5caff0c2decf98b20...,9cbabc969a489d90986ee4dfd642a5caff0c2decf98b20...,CN=3401fa,872377998b24ff65c2fd416a894c1bfaad506e7ba6ac70...,"CN=57,O=Let's Encrypt,C=US"
33653,1.11-7,uva,none,[9cbabc969a489d90986ee4dfd642a5caff0c2decf98b2...,TLS_ECDHE_ECDSA_WITH_AES_128_GCM_SHA256,[5e7970e9e73521806f1755456f899939b0066aa8568c7..

In [54]:
def get_issuer_column(data_path: str, df: str):
        x509_df_path = data_path.replace("ssl", "x509")
        x509_df = SQLContext.read.parquet(x509_df_path)
        x509_df = x509_df.select(["fingerprint", "certificate_issuer"])
        ssl_df = df.withColumn("fingerprint", df.cert_chain_fps[0])
        ssl_df = ssl_df.join(x509_df, on="fingerprint", how="inner")
        ssl_df = ssl_df.withColumnRenamed("certificate_issuer", "issuer")
        return ssl_df

In [55]:
data_path_2 = "/mnt/chaseproject/uva/kd5eyn/ssl/2022-04-30/anon.ssl_20220430_2100-2130-0400.log.gz"
ssl_df_2 = SQLContext.read.parquet(data_path_2)
ssl_df_2 = get_issuer_column(data_path_2, ssl_df_2)

In [57]:
ssl_df_2.count()

9335